**MODELO DE RECOMENDACIÓN**

In [1]:
# Importamos librerias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
#importando datasets
df = pd.read_csv(r'df_final.csv')
df.shape

C:\Users\belen\AppData\Local\Temp\ipykernel_12780\2790493315.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'df_final.csv')


(45379, 20)

Definimos un rango de datos

In [3]:
#Decidi tomar esta desicion ya que al entrenar mi modelo con todos los datos me daba error de memoria, a mi RAM no le da, consulté la situación y me aconsejaron
# Eliminar las columnas que no eran necesarias pero aún así tampoco pude, intente de otras formas más y no lo logré, asi que para presentar algo, tome solo 1000 datos
#Al tener solo esa cantidad no va a funcion de la manera correcta pero el codigo para realizarlo es el correcto(espero).
Df= df.sample(n=1000, random_state=42)

In [4]:
Df.shape

(1000, 20)

In [6]:
#Reseteamos el indice
Df.reset_index(drop=True, inplace=True)

In [7]:
Df['title']

0                    Extraction
1              Print the Legend
2                 The Glass Key
3             The Sunshine Boys
4                      No Mercy
                 ...           
995          The Pick-up Artist
996                    Sky High
997    Bus Riley's Back In Town
998                   Hadewijch
999                     Ed Wood
Name: title, Length: 1000, dtype: object

In [8]:
Df.dropna(subset=["title", "genres", "tagline", "overview"], inplace=True)

Creamos la matriz similar

In [9]:
# Aplicar la técnica TF-IDF para representar los textos en forma numérica
tfidf = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(Df["title"] + " " + str(Df["genres"]) + " " + Df["tagline"] + " " + Df["overview"])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Exportamos el archivo pickle

In [10]:
with open('similarity_matrix.pickle', 'wb') as f:
    pickle.dump(cosine_sim, f)

Creamos la función para evaluar el modelo

In [11]:

# Función que recibe un título de película y devuelve las 5 películas más similares
def recomendacion(titulo):
    # Buscar la fila correspondiente al título de la película
    idx = Df.index[Df["title"].str.lower() == titulo.lower()].tolist()
    if len(idx) == 0:
        return "Película no encontrada"
    else:
        idx = idx[0]
    
    # Calcular la similitud de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar las películas según su similitud y seleccionar las 5 más similares
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    
    # Obtener los índices de las películas recomendadas
    movie_indices = [i[0] for i in sim_scores]
    
    # Devolver los títulos de las películas recomendadas
    return list(Df["title"].iloc[movie_indices])


Ejemplo

In [15]:
Df['title']

0              Extraction
2           The Glass Key
3       The Sunshine Boys
4                No Mercy
6                    Push
              ...        
992             Wild Hogs
993                Choose
995    The Pick-up Artist
996              Sky High
999               Ed Wood
Name: title, Length: 463, dtype: object

In [19]:
recomendacion('Push')

['One A.M.', 'Lovers and Liars', 'Premam', "Hell's Highway", 'So Big!']

Guardamos en un nuevo df para la API

In [20]:
Df.to_csv('Df_recomendacion.csv', index=False)

------------------------------